# Turner Lab M1 MPTP Dataset - NWB Usage Guide

**Dataset Overview:**
This dataset contains single-unit electrophysiology recordings from primary motor cortex (M1) of parkinsonian macaque monkeys performing flexion/extension motor tasks. The data investigates motor encoding deficits in MPTP-induced parkinsonism, comparing pyramidal tract neurons (PTNs) versus corticostriatal neurons (CSNs).

**Key Features:**
- **Single-unit recordings**: Spike times and waveforms from M1 neurons
- **Motor behavior**: Flexion/extension task with analog kinematics
- **Cell type identification**: Antidromic stimulation to classify PTNs vs CSNs
- **Disease model**: MPTP-treated parkinsonian macaque monkeys
- **Electrode mapping**: Systematic cortical penetrations with stereotactic coordinates

**Data Organization:**
- Each NWB file represents one recording session from one penetration depth
- Session IDs follow pattern: `{subject_id}++{FileName}++{PreMPTP|PostMPTP}++Depth{depth_um}um++{YearMonthDay}`
- Example: `V++{v0502}++PostMPTP++Depth19180um++20000121` indicates monkey V, post-MPTP condition, 19.18mm depth, recorded Jan 21, 2000

## Streaming NWB Files from DANDI 

We recommend using the DANDI Python client to access the NWB files directly from the DANDI archive without downloading them locally.

In [ ]:
import h5py
import remfile
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient

# Connect to DANDI and get the dandiset
dandiset_id = "001636"
client = DandiAPIClient()
dandiset = client.get_dandiset(dandiset_id, "draft")

# Get all assets - users can filter this list to select any session
assets = dandiset.get_assets()
assets_list = list(assets)
print(f"Total assets in dandiset: {len(assets_list)}")

# Filter for NWB files only
nwb_assets = [a for a in assets_list if a.path.endswith(".nwb")]
print(f"NWB files: {len(nwb_assets)}")

# We select this specific session because it is "complete" - it has all available data types:
# - 2 PTNs (pyramidal tract neurons) identified via antidromic stimulation
# - EMG recordings from multiple arm muscles
# - LFP data
# - Perturbation trials (52 out of 80 total trials)
# - Balanced flexion/extension movements (40 each)
#
# The only missing feature is receptive field locations (not recorded for these units).
# See dandiset_session_metadata.csv for a full inventory of all 298 sessions.
asset_path = "sub-V/sub-V_ses-V++v5811++PostMPTP++Depth18300um++20000331_behavior+ecephys.nwb"
asset = next(a for a in nwb_assets if a.path == asset_path)
print(f"Selected asset: {asset.path}")

# Stream the NWB file directly from DANDI (no download required)
s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
file_system = remfile.File(s3_url)
file = h5py.File(file_system, mode="r")

io = NWBHDF5IO(file=file)
nwbfile = io.read()

Fist, we can visualize the HTML representation of the nwbfile to get an overview of its contents.

In [ ]:
nwbfile

## Understanding the Trial Structure

The behavioral task is a visuomotor step-tracking paradigm where monkeys make rapid elbow flexion/extension movements to capture visual targets. Each trial follows a stereotyped sequence of events that we can examine through the trials table.

In [ ]:
nwbfile.trials

Trials are stored as a table in NWB. We can convert it to a pandas DataFrame to explore the structure of the trial events

In [ ]:
trials_df = nwbfile.trials.to_dataframe()
trials_df.sample(n=5)

### Trial Event Sequence

Let's examine the key columns that define the temporal structure of each trial:

In [ ]:
columns = [
    "start_time",
    "center_target_appearance_time",
    "lateral_target_appearance_time",
    "cursor_departure_time",
    "movement_type",
    "reward_time",
]
trials_df[columns].head(n=5)

Each trial progresses through distinct phases:

1. **Trial start**: Recording begins with a variable baseline period
2. **Center target appearance**: A center target appears on screen, cueing the monkey to align the cursor and hold
3. **Center hold period**: The monkey maintains position for 1-2 seconds (randomized to prevent anticipation)
4. **Lateral target appearance (go cue)**: A peripheral target appears, signaling the monkey to move
5. **Cursor departure**: The monkey initiates movement, exiting the center zone
6. **Movement execution**: Rapid ballistic movement toward the target (flexion or extension)
7. **Reward**: Liquid reward delivered upon successful target acquisition

The time differences between these events reveal reaction times, movement durations, and other behaviorally relevant measures.

To graphically understand the trial structure, we can look at the following visualization that is using the data in the NWB file:

In [ ]:
from trial_structure_plot import plot_trial_structure

plot_trial_structure(trials_df, max_trials=10);

The visualization above shows the temporal structure of individual trials as horizontal stacked bars. Each row represents one trial, with colored segments indicating different task phases:

- **Baseline** (gray): Variable waiting period before the task begins
- **Holding at Center** (teal): The monkey maintains cursor position at the center target, waiting for the go cue
- **Time to React** (yellow): The interval between go cue (lateral target appearance) and cursor departure from the center zone
- **Movement** (coral): Active movement execution toward the peripheral target
- **Holding at Target** (purple): The monkey holds position at the target before reward delivery
- **Post-Reward** (sky blue): Brief period after reward before trial ends

Trials labeled "Aborted" were terminated early (the monkey broke fixation before the go cue). Notice the consistent structure across trials, with the main variability coming from the randomized center hold duration (1-2 seconds) and individual differences in reaction time and movement speed.

### Derived Kinematic Measures

In addition to trial event times, the trials table includes derived kinematic parameters computed from post-hoc analysis of the elbow angle velocity trace. These measures characterize the movement itself:

| Column | Description | Units |
|--------|-------------|-------|
| `derived_movement_onset_time` | Time when movement began (detected from velocity trace) | seconds |
| `derived_movement_end_time` | Time when movement ended (velocity returned to baseline) | seconds |
| `derived_peak_velocity` | Maximum angular velocity during the movement | degrees/second |
| `derived_peak_velocity_time` | Time of peak velocity | seconds |
| `derived_movement_amplitude` | Total angular displacement (end position - start position) | degrees |
| `derived_end_position` | Final elbow angle at movement end | degrees |

The "derived" prefix indicates these values were computed offline from kinematic analysis, not recorded directly during the experiment. The detection algorithm used position, velocity, and duration criteria (exact parameters not documented in source data).

**Sign conventions:**
- Positive velocities and amplitudes indicate extension movements
- Negative velocities and amplitudes indicate flexion movements

In [ ]:
# Access derived kinematic measures
kinematic_columns = [
    'derived_movement_onset_time',
    'derived_movement_end_time',
    'derived_peak_velocity',
    'derived_peak_velocity_time',
    'derived_movement_amplitude',
    'derived_end_position',
    'movement_type'
]
trials_df[kinematic_columns].head(n=5)

In [ ]:
# Compute commonly used derived measures
# Note: Some trials may have NaN values if kinematic detection failed

# Movement duration (from movement onset to movement end)
trials_df['movement_duration'] = (
    trials_df['derived_movement_end_time'] - trials_df['derived_movement_onset_time']
)

# Reaction time (from go cue to cursor departure)
trials_df['reaction_time'] = (
    trials_df['cursor_departure_time'] - trials_df['lateral_target_appearance_time']
)

# Filter to complete trials only
complete_trials = trials_df.dropna(subset=['derived_movement_end_time'])

print(f"Complete trials with kinematic data: {len(complete_trials)} / {len(trials_df)}")
print(f"\nMovement duration: {complete_trials['movement_duration'].mean()*1000:.0f} +/- {complete_trials['movement_duration'].std()*1000:.0f} ms")
print(f"Reaction time: {complete_trials['reaction_time'].mean()*1000:.0f} +/- {complete_trials['reaction_time'].std()*1000:.0f} ms")
print(f"Peak velocity: {complete_trials['derived_peak_velocity'].abs().mean():.1f} +/- {complete_trials['derived_peak_velocity'].abs().std():.1f} deg/s")

### Torque Perturbation Trials

A subset of trials included brief mechanical perturbations delivered via the torque motor during the center hold period. These perturbations caused rapid muscle stretch, allowing researchers to study proprioceptive responses in M1 neurons (see Pasquereau & Turner, 2013).

**Note:** Not all sessions include perturbation trials. Sessions without perturbations will have `torque_perturbation_type = "none"` for all trials and `torque_perturbation_onset_time = NaN`.

| Column | Description | Values |
|--------|-------------|--------|
| `torque_perturbation_type` | Direction of the perturbation | `"flexion"`, `"extension"`, or `"none"` |
| `torque_perturbation_onset_time` | Time when perturbation was delivered | seconds (NaN if no perturbation) |

**Perturbation characteristics (when present):**
- Delivered 150-500 ms after center target appearance (during hold period)
- Brief torque pulse causing ~5-10 degree displacement
- Direction (flexion/extension) was independent of the upcoming movement direction
- Present in approximately 65% of trials in sessions that included perturbations

**Scientific purpose:** These trials enabled analysis of how M1 neurons respond to passive muscle stretch, separate from active movement. The muscle stretch study found that PTNs showed degraded directional selectivity to perturbations in the parkinsonian state.

In [ ]:
# Access torque perturbation data
perturbation_columns = ['torque_perturbation_type', 'torque_perturbation_onset_time', 'movement_type']
trials_df[perturbation_columns].head(n=10)

In [ ]:
# Analyze perturbation trial distribution
# Note: Not all sessions include perturbation trials. Sessions without perturbations
# will have torque_perturbation_type = "none" for all trials.

perturbation_counts = trials_df['torque_perturbation_type'].value_counts()
n_perturbation_trials = len(trials_df[trials_df['torque_perturbation_type'] != 'none'])

# Check if this session has perturbation data
if n_perturbation_trials == 0:
    print("This session does not include torque perturbation trials.")
    print("All trials have torque_perturbation_type = 'none'")
else:
    print(f"This session includes perturbation trials ({n_perturbation_trials} / {len(trials_df)} trials)\n")
    print("Perturbation trial counts:")
    for ptype, count in perturbation_counts.items():
        print(f"  {ptype}: {count} trials ({100*count/len(trials_df):.1f}%)")
    
    # Cross-tabulate perturbation direction vs movement direction
    print("\nPerturbation direction vs Movement direction:")
    print(trials_df.groupby(['torque_perturbation_type', 'movement_type']).size().unstack(fill_value=0))

### Isolation Monitoring Stimulation

In some recording sessions, single antidromic stimulation pulses were delivered early in certain trials to activate otherwise silent neurons and monitor unit isolation. This was particularly important for intratelencephalic-type (IT) cortical neurons (including corticostriatal neurons), which have very low spontaneous firing rates.

**Background:** Some neurons rarely spike spontaneously but can be reliably activated by antidromic stimulation from their projection target (striatum, thalamus, or cerebral peduncle). Delivering occasional stimulation pulses during recording confirmed that the unit remained well-isolated throughout the session.

| Column | Description | Values |
|--------|-------------|--------|
| `isolation_monitoring_stim_time` | Time when stimulation pulse was delivered | seconds (NaN if no stimulation in that trial) |
| `isolation_monitoring_stim_site` | Anatomical target of the stimulation | `"Str"` (striatum), `"Thal"` (thalamus), `"Ped"` (peduncle), or empty string |

**Characteristics:**
- Stimulation pulses delivered ~244-245 ms after trial start (early in the baseline period)
- Present in approximately 43% of sessions (171 out of 400 session files)
- Most commonly targeting thalamus (117 sessions), followed by striatum (51 sessions)
- When present, stimulation typically delivered in a subset of trials within the session

**Note:** If you observe spike times that appear suspiciously constant across trials (~245-256 ms after trial start), these likely reflect antidromically-evoked spikes from isolation monitoring, not spontaneous activity.

In [ ]:
# Access isolation monitoring stimulation data (only trials with stimulation)
stim_columns = ['start_time', 'isolation_monitoring_stim_time', 'isolation_monitoring_stim_site']
stim_trials = trials_df[trials_df['isolation_monitoring_stim_time'].notna()]
stim_trials[stim_columns].head(n=10)

In [ ]:
# Check if this session has isolation monitoring stimulation
stim_trials = trials_df[trials_df['isolation_monitoring_stim_time'].notna()]
n_stim_trials = len(stim_trials)

if n_stim_trials == 0:
    print("This session does not include isolation monitoring stimulation.")
    print("All trials have isolation_monitoring_stim_time = NaN")
else:
    print(f"This session includes isolation monitoring stimulation ({n_stim_trials} / {len(trials_df)} trials)\n")
    
    # Get stimulation site (should be consistent within session)
    stim_site = stim_trials['isolation_monitoring_stim_site'].iloc[0]
    print(f"Stimulation site: {stim_site}")
    
    # Calculate timing relative to trial start
    stim_trials = stim_trials.copy()
    stim_trials['stim_time_relative'] = (
        stim_trials['isolation_monitoring_stim_time'] - stim_trials['start_time']
    )
    
    mean_stim_time = stim_trials['stim_time_relative'].mean() * 1000  # Convert to ms
    std_stim_time = stim_trials['stim_time_relative'].std() * 1000
    
    print(f"Stimulation timing: {mean_stim_time:.1f} +/- {std_stim_time:.1f} ms after trial start")

## Accessing Neurons/Units Spikes

The units table contains spike times and metadata for each recorded neuron. A key feature of this dataset is the classification of neurons into pyramidal tract neurons (PTNs) and corticostriatal neurons (CSNs) through antidromic stimulation.

In [ ]:
nwbfile.units

Like the trials table, the units table is a DynamicTable that can be converted to a pandas DataFrame for easier exploration and analysis.

In [ ]:
units_df = nwbfile.units.to_dataframe()
units_df

### Accessing Spike Times

Each unit's spike times are stored as an array of timestamps (in seconds) relative to session start. You can access them by unit index.

In [ ]:
# Accessing spike times
for unit_id in range(len(units_df)):
    spike_times = nwbfile.units['spike_times'][unit_id]
    
    print(f"Unit {unit_id}:")
    print(f"  Total spikes: {len(spike_times)}")
    print(f"  Recording duration: {spike_times[-1] - spike_times[0]:.2f} s")
    print(f"  Mean firing rate: {len(spike_times) / (spike_times[-1] - spike_times[0]):.2f} Hz")
    print()

In [ ]:
# Simple spike raster plot
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(12, 3))

for unit_id in range(len(units_df)):
    spike_times = nwbfile.units['spike_times'][unit_id]
    ax.scatter(spike_times, np.ones_like(spike_times) * unit_id, 
               s=0.5, color='black', alpha=0.6)

ax.set_xlabel('Time (s)')
ax.set_ylabel('Unit')
ax.set_yticks(range(len(units_df)))
ax.set_title('Spike Raster')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()
plt.show()

### Neuron Projection Type (Cell Classification)

Each neuron in this dataset was classified by its axonal projection target using antidromic stimulation. This classification allows you to analyze PTNs and CSNs separately.

| Column | Description | Values |
|--------|-------------|--------|
| `neuron_projection_type` | Classification based on antidromic response | `"pyramidal_tract_neuron"`, `"corticostriatal_neuron"`, `"no_response"`, `"not_tested"` |

**Cell type definitions:**

- **Pyramidal Tract Neurons (PTNs)**: Large layer 5b neurons whose axons descend through the cerebral peduncle to the spinal cord. These are the primary motor output neurons that send commands directly to spinal motor circuits. Identified by antidromic activation from the cerebral peduncle.

- **Corticostriatal Neurons (CSNs)**: Intratelencephalic-type layer 5 neurons projecting to the striatum (putamen). These provide cortical input to basal ganglia circuits involved in action selection. Identified by antidromic activation from the posterolateral putamen.

- **No Response**: Neuron was tested but did not respond antidromically to any stimulation site.

- **Not Tested**: Antidromic testing was not performed for this neuron.

In [ ]:
# Examine neuron classification
classification_columns = ['neuron_projection_type', 'is_post_mptp', 'unit_name']
units_df[classification_columns]

### Antidromic Identification Parameters

For neurons that responded antidromically, the units table includes parameters from the identification protocol:

| Column | Description | Units |
|--------|-------------|-------|
| `antidromic_stimulation_sites` | Anatomical site(s) where stimulation evoked antidromic response | `"cerebral_peduncle"`, `"putamen"`, or both |
| `antidromic_latency_ms` | Response latency from stimulation to spike | milliseconds |
| `antidromic_threshold` | Minimum current required to evoke response | microamperes |
| `antidromic_latency_2_ms` | Secondary site latency (for dual-projection neurons) | milliseconds |
| `antidromic_threshold_2` | Secondary site threshold | microamperes |

**Latency interpretation:**
- **PTNs (peduncle stimulation)**: 0.6-2.5 ms latencies indicate large-diameter, fast-conducting axons (~20-50 m/s)
- **CSNs (putamen stimulation)**: 2-20 ms latencies indicate smaller, slower-conducting axons typical of intratelencephalic neurons

In [ ]:
# Examine antidromic identification parameters
antidromic_columns = [
    'neuron_projection_type',
    'antidromic_stimulation_sites',
    'antidromic_latency_ms',
    'antidromic_threshold'
]
units_df[antidromic_columns]

### Receptive Field Properties

During recording sessions, experimenters performed qualitative sensory testing to characterize each neuron's receptive field. This involved manually manipulating the monkey's arm while listening to the neural activity.

| Column | Description | Values |
|--------|-------------|--------|
| `receptive_field_location` | Body region(s) comprising the receptive field | `"hand"`, `"wrist"`, `"elbow"`, `"shoulder"`, combinations, `"no_response"`, or `"not_tested"` |
| `receptive_field_stimulus` | Type of manipulation that activated the neuron | `"ext"`, `"flex"`, `"pron"`, `"light touch"`, `"active grip"`, etc. |

**Common abbreviations in `receptive_field_stimulus`:**
- `ext` = extension, `flex` = flexion
- `pron` = pronation, `sup` = supination
- `cut` = cutaneous (skin stimulation)
- `NR` = no response, `NT` = not tested

**Interpretation:**
- Most neurons responded to distal arm regions (hand, wrist, fingers), consistent with recording in arm-related M1
- Multiple regions separated by ` / ` indicate neurons with broad receptive fields
- Sensory characterization is independent of but complementary to antidromic cell-type identification

In [ ]:
# Examine receptive field properties
rf_columns = ['receptive_field_location', 'receptive_field_stimulus', 'neuron_projection_type']
units_df[rf_columns]

In [ ]:
# Examine receptive field properties (units with responses)
rf_columns = ['receptive_field_location', 'receptive_field_stimulus', 'neuron_projection_type']
units_with_rf = units_df[~units_df['receptive_field_location'].isin(['not_tested', 'no_response'])]

if len(units_with_rf) > 0:
    print(f"Units with receptive field responses: {len(units_with_rf)} / {len(units_df)}")
    display(units_with_rf[rf_columns])
else:
    print("No units in this session have documented receptive field responses.")
    print("Showing all units:")
    display(units_df[rf_columns])

### Additional Unit Metadata

| Column | Description | Values |
|--------|-------------|--------|
| `is_post_mptp` | Whether recording was obtained after MPTP treatment | `True` or `False` |
| `unit_also_in_session_id` | Session ID if this unit also appears in another file | session ID string or empty |

**Note on `unit_also_in_session_id`:** Some neurons were recorded across multiple electrode depths within the same penetration. When a unit appears in multiple session files, this column contains the session ID of the related file. This allows linking the same neuron across different recording depths.

### Electrodes Table

The electrodes table contains metadata for the recording electrode, including chamber-relative coordinates that allow mapping recordings to anatomical locations in M1.

| Column | Description | Units/Values |
|--------|-------------|--------------|
| `chamber_grid_ap_mm` | Anterior-posterior position in chamber grid | mm (positive = anterior, range: -7 to +6) |
| `chamber_grid_ml_mm` | Medial-lateral position in chamber grid | mm (positive = lateral, range: -6 to +2) |
| `chamber_insertion_depth_mm` | Depth from chamber surface | mm (range: 8.4-27.6) |

**Recording electrode:** Glass-coated platinum-iridium microelectrode lowered through the recording chamber into M1. The chamber coordinates allow reconstructing electrode positions relative to the cortical sulcal pattern.

**Coordinate system:** Chamber-relative coordinates specific to each animal's implanted recording chamber. The chamber was positioned over left M1 at 35 degrees to the coronal plane. These are NOT standard stereotaxic atlas coordinates.

**Note:** Stimulation electrodes (for antidromic identification) are represented as separate devices, not in the electrodes table. See `nwbfile.devices` for stimulation electrode descriptions.

In [ ]:
# Access electrodes table
electrodes_df = nwbfile.electrodes.to_dataframe()
electrodes_df

In [ ]:
# Get recording electrode coordinates
recording_electrode = electrodes_df.iloc[0]

print("Recording electrode location:")
print(f"  Chamber A/P: {recording_electrode['chamber_grid_ap_mm']:.1f} mm")
print(f"  Chamber M/L: {recording_electrode['chamber_grid_ml_mm']:.1f} mm")
print(f"  Depth: {recording_electrode['chamber_insertion_depth_mm']:.2f} mm")

## Analog Signals (Kinematics, EMG, LFP)

The acquisition container holds continuous time series data recorded during the task. These signals are concatenated across trials with the same timestamps as the trials table.

In [ ]:
nwbfile.acquisition

### Manipulandum Signals (Kinematics)

The monkey controlled a cursor by rotating a manipulandum with their elbow. Three kinematic signals are recorded:

| Signal | Location | Description | Units |
|--------|----------|-------------|-------|
| `ElbowAngle` | `acquisition` | Elbow joint angle | degrees |
| `ElbowVelocity` | `processing['behavior']` | Angular velocity (derivative of position) | degrees/second |
| `ElbowTorque` | `acquisition` | Torque command sent to the motor | arbitrary units |

**Sign conventions:**
- Positive values indicate extension direction
- Negative values indicate flexion direction

In [ ]:
# Access kinematic signals
elbow_angle = nwbfile.acquisition['ElbowAngle']
elbow_velocity = nwbfile.processing['behavior']['ElbowVelocity']

print(f"Elbow Angle: {elbow_angle.data.shape[0]} samples")
print(f"Elbow Velocity: {elbow_velocity.data.shape[0]} samples")

# Get data and timestamps
angle_data = elbow_angle.data[:]
velocity_data = elbow_velocity.data[:]
timestamps = elbow_angle.timestamps[:]

print(f"Duration: {timestamps[-1] - timestamps[0]:.1f} s")

In [ ]:
# Plot kinematic signals for a single trial
trial_idx = 5
trial = trials_df.iloc[trial_idx]
t_start, t_stop = trial['start_time'], trial['stop_time']

# Extract trial segment using timestamps
mask = (timestamps >= t_start) & (timestamps <= t_stop)
trial_time = timestamps[mask] - t_start  # Relative to trial start

fig, axes = plt.subplots(2, 1, figsize=(10, 5), sharex=True)

axes[0].plot(trial_time, angle_data[mask], 'b-', linewidth=1)
axes[0].set_ylabel('Angle (deg)')
axes[0].set_title(f'Trial {trial_idx} - {trial["movement_type"].capitalize()} Movement')
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)

axes[1].plot(trial_time, velocity_data[mask], 'r-', linewidth=1)
axes[1].set_ylabel('Velocity (deg/s)')
axes[1].set_xlabel('Time (s)')
axes[1].axhline(0, color='gray', linestyle='--', alpha=0.5)
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

### EMG Signals

Electromyography (EMG) recordings from arm muscles are available in some sessions. Each muscle has its own TimeSeries named `EMG{MuscleName}`.

**Note:** Not all sessions include EMG data. Check `nwbfile.acquisition` for available signals.

In [ ]:
# Find EMG signals in acquisition
emg_signals = {name: ts for name, ts in nwbfile.acquisition.items() if name.startswith('EMG')}

if emg_signals:
    print(f"EMG signals available ({len(emg_signals)} muscles):")
    for name, ts in emg_signals.items():
        muscle_name = name.replace('EMG', '')
        print(f"  {muscle_name}: {ts.description}")
else:
    print("No EMG signals in this session.")

In [ ]:
# Plot EMG signals for a single trial (if available)
if emg_signals:
    trial_idx = 5
    trial = trials_df.iloc[trial_idx]
    t_start, t_stop = trial['start_time'], trial['stop_time']
    
    # Get first EMG signal for plotting
    emg_name = list(emg_signals.keys())[0]
    emg_ts = emg_signals[emg_name]
    emg_data = emg_ts.data[:]
    emg_timestamps = emg_ts.timestamps[:]
    
    # Extract trial segment
    mask = (emg_timestamps >= t_start) & (emg_timestamps <= t_stop)
    
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(emg_timestamps[mask] - t_start, emg_data[mask], 'g-', linewidth=0.5)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('EMG (a.u.)')
    ax.set_title(f'Trial {trial_idx} - {emg_name}')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.tight_layout()
    plt.show()

### LFP Signal

Local field potential (LFP) recorded from the same electrode as the single units. Bandpass filtered 1-100 Hz.

**Note:** Not all sessions include LFP data.

In [ ]:
# Check for LFP signal in processing module
if 'ecephys' in nwbfile.processing and 'LFP' in nwbfile.processing['ecephys'].data_interfaces:
    lfp_ts = nwbfile.processing['ecephys']['LFP']
    print(f"LFP signal available: {lfp_ts.data.shape[0]} samples")
    print(f"Units: {lfp_ts.unit}")
    print(f"Description: {lfp_ts.description}")
    
    # Plot LFP for a single trial
    # Note: Data are uncalibrated A/D converter values (centered ~2048), not volts
    trial_idx = 5
    trial = trials_df.iloc[trial_idx]
    t_start, t_stop = trial['start_time'], trial['stop_time']
    
    lfp_data = lfp_ts.data[:]
    lfp_timestamps = lfp_ts.timestamps[:]
    mask = (lfp_timestamps >= t_start) & (lfp_timestamps <= t_stop)
    
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(lfp_timestamps[mask] - t_start, lfp_data[mask], 'purple', linewidth=0.5)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('LFP (a.u.)')
    ax.set_title(f'Trial {trial_idx} - LFP')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.tight_layout()
    plt.show()
else:
    print("No LFP signal in this session.")

## Antidromic Stimulation Raw Waveforms

In addition to the classification results in the units table, the raw antidromic stimulation data is available in the `AntidromicSweepsIntervals` table. Each row represents one 50ms sweep containing both the neural response and stimulation current waveforms.

| Column | Description | Values |
|--------|-------------|--------|
| `start_time` | Sweep start time (25ms before stimulation) | seconds |
| `stop_time` | Sweep end time (25ms after stimulation) | seconds |
| `stimulation_onset_time` | Time of stimulation pulse delivery (t=0 in sweep) | seconds |
| `unit_name` | Unit number from source filename | `"1"` or `"2"` |
| `location` | Stimulation site | `"Striatum"`, `"Peduncle"`, `"Thalamus"` |
| `stimulation_protocol` | Test type performed | `"Collision"`, `"FrequencyFollowing"`, `"Orthodromic"` |
| `sweep_number` | Sweep index within the stimulation series | integer |
| `response` | Reference to neural voltage ElectricalSeries | object reference |
| `stimulation` | Reference to stimulation current TimeSeries | object reference |

**Protocol types explained:**
- **Collision**: Tests whether a spontaneous spike traveling down the axon blocks the antidromic spike - the gold-standard verification for antidromic activation
- **FrequencyFollowing**: Tests whether the neuron can follow high-frequency stimulation (>200 Hz) with consistent latency - verifies direct axonal activation vs synaptic
- **Orthodromic**: Control stimulation that activates the neuron through synaptic pathways (sub-threshold for antidromic)

**Note:** Not all sessions include antidromic stimulation data. Check if the intervals table exists before accessing.

In [ ]:
# Check if antidromic sweeps data exists and explore the table
if "AntidromicSweepsIntervals" in nwbfile.intervals:
    sweeps_df = nwbfile.intervals["AntidromicSweepsIntervals"].to_dataframe()
    print(f"Antidromic sweeps available: {len(sweeps_df)} sweeps")
    print(f"\nSweeps by stimulation location:")
    print(sweeps_df['location'].value_counts())
    print(f"\nSweeps by protocol type:")
    print(sweeps_df['stimulation_protocol'].value_counts())
else:
    print("No antidromic sweeps data in this session.")

In [ ]:
# Preview the sweeps table structure
if "AntidromicSweepsIntervals" in nwbfile.intervals:
    display_columns = ['unit_name', 'location', 'stimulation_protocol', 'sweep_number', 
                       'stimulation_onset_time']
    sweeps_df[display_columns].head(10)

### Accessing Raw Antidromic Waveforms

Each sweep contains two linked time series:
- **response**: Neural voltage recording from the M1 electrode (in volts)
- **stimulation**: Stimulation current delivered to the target site (in amperes)

The sweeps are 50ms long (1000 samples at 20kHz), centered on the stimulation pulse. To extract waveforms aligned to stimulation onset:

In [ ]:
# Extract and plot a single antidromic sweep
if "AntidromicSweepsIntervals" in nwbfile.intervals:
    # Get the first sweep
    sweep = sweeps_df.iloc[0]
    
    # Access the response and stimulation data through TimeSeriesReference
    # Each reference is a tuple: (idx_start, count, timeseries)
    response_ref = sweep['response']
    index_start, count, response_series = response_ref
    response_data = response_series.data[index_start:index_start + count].flatten() * response_series.conversion * 1e6  # uV
    
    stim_ref = sweep['stimulation']
    index_start_s, count_s, stim_series = stim_ref
    stim_data = stim_series.data[index_start_s:index_start_s + count_s].flatten() * stim_series.conversion * 1e6  # uA
    
    # Create time axis relative to stimulation onset
    # Data starts 25ms before stimulation (t=0 is at sample 500 for 20kHz)
    sampling_rate = response_series.rate
    time_ms = (np.arange(count) / sampling_rate - 0.025) * 1000  # Time in ms relative to stim
    
    # Plot
    fig, axes = plt.subplots(2, 1, figsize=(10, 5), sharex=True)
    
    axes[0].plot(time_ms, response_data, 'b-', linewidth=0.8)
    axes[0].axvline(0, color='red', linestyle='--', alpha=0.7, label='Stimulation')
    axes[0].set_ylabel('Neural Response (uV)')
    axes[0].set_title(f"Antidromic Sweep - Unit {sweep['unit_name']}, {sweep['location']}, {sweep['stimulation_protocol']}")
    axes[0].legend(loc='upper right')
    axes[0].spines['top'].set_visible(False)
    axes[0].spines['right'].set_visible(False)
    
    axes[1].plot(time_ms, stim_data, 'r-', linewidth=0.8)
    axes[1].axvline(0, color='red', linestyle='--', alpha=0.7)
    axes[1].set_ylabel('Stim Current (uA)')
    axes[1].set_xlabel('Time relative to stimulation (ms)')
    axes[1].spines['top'].set_visible(False)
    axes[1].spines['right'].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nSweep metadata:")
    print(f"  Stimulation site: {sweep['location']}")
    print(f"  Protocol: {sweep['stimulation_protocol']}")
    print(f"  Sampling rate: {sampling_rate} Hz")

In [ ]:
# Overlay multiple collision test sweeps to visualize response consistency
if "AntidromicSweepsIntervals" in nwbfile.intervals:
    # Filter to collision test sweeps only
    collision_sweeps = sweeps_df[sweeps_df['stimulation_protocol'] == 'Collision']
    
    if len(collision_sweeps) > 0:
        fig, ax = plt.subplots(figsize=(10, 4))
        
        # Plot first 20 sweeps (or all if fewer)
        n_to_plot = min(20, len(collision_sweeps))
        
        for i in range(n_to_plot):
            sweep = collision_sweeps.iloc[i]
            
            # Access data through TimeSeriesReference
            response_ref = sweep['response']
            index_start, count, response_series = response_ref
            response_data = response_series.data[index_start:index_start + count].flatten() * response_series.conversion * 1e6
            
            sampling_rate = response_series.rate
            time_ms = (np.arange(count) / sampling_rate - 0.025) * 1000
            
            ax.plot(time_ms, response_data, 'b-', linewidth=0.5, alpha=0.5)
        
        ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Stimulation')
        ax.set_xlabel('Time relative to stimulation (ms)')
        ax.set_ylabel('Neural Response (uV)')
        ax.set_title(f'Collision Test Sweeps Overlay (n={n_to_plot})')
        ax.legend(loc='upper right')
        ax.set_xlim(-5, 15)  # Focus on the response window
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        plt.tight_layout()
        plt.show()
        
        # Get the antidromic latency from the units table for comparison
        unit_name = collision_sweeps.iloc[0]['unit_name']
        unit_row = units_df[units_df['unit_name'] == unit_name]
        if len(unit_row) > 0:
            latency = unit_row['antidromic_latency_ms'].values[0]
            if latency == latency:  # Check for NaN (NaN != NaN is True)
                print(f"Expected antidromic response at {latency:.2f} ms (from units table)")
    else:
        print("No collision test sweeps found in this session.")

### Stimulation Electrodes Table

The `StimulationElectrodesTable` in the `antidromic_identification` processing module documents the chronically implanted electrodes used for antidromic testing:

| Index | Location | Purpose |
|-------|----------|---------|
| 0 | Cerebral peduncle | PTN identification (pyramidal tract) |
| 1-3 | Posterolateral putamen | CSN identification (3 electrodes) |
| 4 | Ventrolateral thalamus | Thalamocortical projection identification |

These are distinct from the recording microelectrode in the electrodes table - they are macroelectrodes surgically implanted at fixed locations for stimulation, not recording.

In [ ]:
# Access the stimulation electrodes table
if "antidromic_identification" in nwbfile.processing:
    antidromic_module = nwbfile.processing["antidromic_identification"]
    if "StimulationElectrodesTable" in antidromic_module.data_interfaces:
        stim_electrodes = antidromic_module["StimulationElectrodesTable"].to_dataframe()
        display(stim_electrodes)
    else:
        print("StimulationElectrodesTable not found in this session.")
else:
    print("No antidromic_identification processing module in this session.")